In [1]:
from utils import MongoConnectionManager
from pprint import pprint
import os

In [2]:
def build_match_query(query_parameters):
    match_query = {}
    
    for key, value in query_parameters.items():
        field_name = key.upper()
        if type(value) == list:
            match_query[field_name] = {'$in': value}
        else:
            match_query[field_name] = {
                '$gte': float(value['min']),
                '$lte': float(value['max'])
            }
    
    return match_query

In [3]:
def extend_geoquery(query, query_parameters):
    for key, value in query_parameters.items():
        if key in {'latitude', 'longitude'}:
            continue
        field_name = key.upper()
        if type(value) == list:
            query[field_name] = {'$in': value}
        else:
            query[field_name] = {
                '$gte': float(value['min']),
                '$lte': float(value['max'])
            }
    
    return query

In [4]:
def build_table_sort_query(sort_parameters, geo_query=False):
    sort_fields = sort_parameters['sort_fields'][::-1]
    sort_directions = sort_parameters['sort_directions'][::-1]
    
    if geo_query:
        sort_query = list()
    else:
        sort_query = dict()
    for field, direction in zip(sort_fields, sort_directions):
        if geo_query:
            sort_query.append((field.upper(), int(direction)))
        else:
            sort_query[field.upper()] = int(direction)
    
    return sort_query

In [5]:
def get_table_data(section_name, query_parameters, page_number, sort_parameters):
    match_query = build_match_query(query_parameters)
    pipeline = [{'$match': match_query}]
    
    if sort_parameters:
        sort_query = build_table_sort_query(sort_parameters)
        pipeline.append({'$sort': sort_query})
    
    offset = (page_number - 1) * 50
    pipeline.extend([
        {'$skip': offset},
        {'$limit': 50},
        {'$project': {'_id': 0}}
    ])
    
    database = os.environ.get('DATABASE')
    collection_name = f'{section_name}_collection'
    with MongoConnectionManager(database, collection_name) as collection:
        data = list(collection.aggregate(pipeline=pipeline, allowDiskUse=True))
    
    return data

In [6]:
def get_map_based_table_data(section_name, query_parameters, page_number, sort_parameters, polygon):
    query = {
        'geometry': {
            '$geoWithin': {
                '$geometry': {
                    'type': 'Polygon',
                    'coordinates': [polygon]
                }
            }
        }
    }
    
    if query_parameters:
        query = extend_geoquery(query, query_parameters)
    
    offset = (page_number - 1) * 50
    sort_query = None
    if sort_parameters:
        sort_query = build_table_sort_query(sort_parameters, geo_query=True)
    
    database = os.environ.get('DATABASE')
    collection_name = f'{section_name}_collection'
    with MongoConnectionManager(database, collection_name) as collection:
        if sort_query:
            data = list(collection.find(query, {'_id': 0}).sort(sort_query).skip(offset).limit(50))
        else:
            data = list(collection.find(query, {'_id': 0}).skip(offset).limit(50))
    
    return data

In [7]:
def format_table_data(data):
    result = []
    for doc in data:
        temp = {key.lower(): value for key, value in doc.items()}
        result.append(temp)
    
    return result

In [8]:
def prepare_table_response(section_name, payload):
    page_number = payload.get('page', 1)
    sort_parameters = payload.get('sort_parameters', None)
    query_parameters = payload.get('query_parameters', {})
    polygon = payload.get('polygon', None)
    
    if polygon:
        data = get_map_based_table_data(section_name, query_parameters, page_number, sort_parameters, polygon)
    else:
        data = get_table_data(section_name, query_parameters, page_number, sort_parameters)
    
    response = format_table_data(data)
    return response

In [9]:
sample_query = {
    'query_parameters': {'neighbourhood_code': ['BU', 'BF', 'MF']},
    'page': 2,
    'sort_parameters': {
        'sort_fields':['floor_area', 'construction_year'],
        'sort_directions':['1', '-1']
    },
    'polygon': [
        [10.424374289351274, 60.197409458606614],
        [11.204298074330882, 60.197409458606614],
        [11.204298074330882, 59.67422595168185],
        [10.424374289351274, 59.67422595168185],
        [10.424374289351274, 60.197409458606614]
    ]
}

In [10]:
section_name = os.environ.get('SECTION')
data = prepare_table_response(section_name, sample_query)

In [ ]:
pprint(data)